In [0]:
from google.colab import drive
drive.mount('/gdrive',force_remount = True)
import cv2
import glob
import numpy as np
from keras import callbacks
import keras.backend as K
from sklearn.metrics import roc_auc_score
import os
import pandas as pd
import ast
import random
import keras
import math
from keras.layers.normalization import BatchNormalization
from sklearn import metrics
%matplotlib inline
from keras.optimizers import SGD,Adam
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from keras.applications.resnet50 import preprocess_input
from keras.models import Sequential,Model
from keras.models import model_from_json
from keras.layers import Dense,ZeroPadding2D,Convolution2D,MaxPooling2D,Dropout,Flatten,Activation,Input,concatenate
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
%cd /gdrive
# Most Recent One
!pip install git+https://github.com/rcmalli/keras-vggface.git
# Release Version
!pip install keras_vggface
from keras_vggface.vggface import VGGFace

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


Using TensorFlow backend.


/gdrive
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-tkgyxe7g
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-tkgyxe7g
  Stored in directory: /tmp/pip-ephem-wheel-cache-2c4jbkdf/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


**Making the dataset**

In [0]:
path = '/gdrive/My Drive/data-kaggle/train/'
k = 0
farr = []
for fid in os.listdir(path):
  marr = []
  marr.append(k)
  marr.append(fid)
  path1 = os.path.join(path,fid)
  for mid in os.listdir(path1):
    t = []
    t.append(mid)
    path2 = os.path.join(path1,mid)
    if (os.listdir(path2) == []):
      continue
    for pid in os.listdir(path2):
      t.append(pid)
    marr.append(t)
  farr.append(marr)
  k +=1
posanc = []
#imgp1 = path1 + '/' + farr[j+2]
for i in range(len(farr)):
  pathf = path + farr[i][1] + '/'
  l = len(farr[i])-2
  t2 = []
  t3 = []
  for j in range(l):
    pathm1 = pathf + farr[i][j+2][0] + '/'
    l1  = len(farr[i][j+2])-1
    for c in range(j+1,l):
      l2 = len(farr[i][c+2])-1
      pathm2 = pathf + farr[i][c+2][0] + '/'
      for k in range(l1):
        pathp1 = pathm1 + farr[i][j+2][k+1]
        for g in range(l2):
          pathp2 = pathm2 + farr[i][c+2][g+1]
          temp = []
          temp.append(pathp1)
          temp.append(pathp2)
          t2.append(temp)
          del(temp)
  t3.append(i)
  t3.append(farr[i][1])
  t3.append(t2)
  posanc.append(t3)
  del(t3)
  del(t2)
s =0
for i in range(len(posanc)):
  s += len(posanc[i][2])
import random
posancneg = []
l = len(posanc)
for i in range(l):
  for j in range(len(posanc[i][2])):
    if(i<l/2):
      c = random.randint(i+1,l-1)
    else:
      c = random.randint(0,i-1)
    if (len(farr[c])-1)==1:
      f =c;
      break;
      f1 = m;
    m = random.randint(2,len(farr[c])-1) 
    if len(farr[c][m])-1==0:
      f =c;
      f1 = m;
      break;
    p = random.randint(1,len(farr[c][m])-1)
    path3 = path + farr[c][1] + '/' + farr[c][m][0] + '/' + farr[c][m][p]
    posanc[i][2][j].append(path3)
    
    
    
train_all=[]
for i in range(len(posanc)):
  for j in range(len(posanc[i][2])):
    train_all.append(posanc[i][2][j])


file=open('/gdrive/My Drive/data-kaggle/outputHard.txt','r')
if file.mode == 'r':
  contents =file.read()
images=ast.literal_eval(contents)
    
len_val=200


train=[]
for j in random.sample(range(0,410182), 200000):
  train.append(train_all[j])

 

train_anchor=[]
train_positive=[]
train_negative=[]
for i in range(len(train)):
  train_anchor.append(train[i][0])
  train_positive.append(train[i][1])
  train_negative.append(train[i][2])
  

validation=[]
    
for j in random.sample(range(0,410182), 3000):
  validation.append(train_all[j])

    

val_set=[]
for i in range(len(validation)):
  a=random.randint(1,2)
  b=[]
  b.append(validation[i][0])
  
  if a==1:
    b.append(validation[i][1])
    b.append(1)
  else:
    b.append(validation[i][2])
    b.append(0)
  val_set.append(b)

**Image Alignment**

In [0]:
def create_base_network(in_dims):
  nb_class = 128
  hidden_dim = 512

  vgg_model = VGGFace(weights='vggface',include_top=True, input_shape=(224, 224, 3))
  last_layer = vgg_model.get_layer('fc6/relu').output


  out = Dense(nb_class, activation='tanh', name='fc8')(x)
  custom_vgg_model = Model(vgg_model.input, out)
  
  for layer in custom_vgg_model.layers[:-6]:
    layer.trainable = False
      

  return custom_vgg_model


In [0]:
model=create_base_network((224,224,3))
model.summary()

E0704 12:50:21.414760 139627899881344 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-a0cdceaaaec8>", line 1, in <module>
    model=create_base_network((224,224,3))
  File "<ipython-input-15-9a0ed95efeeb>", line 9, in create_base_network
    out = Dense(nb_class, activation='tanh', name='fc8')(x)
NameError: name 'x' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    

NameError: ignored

**Triplet Loss Function**

In [0]:
from keras import backend as K
def triplet_loss(y_true,y_pred, alpha = 0.4):
    total_lenght = y_pred.shape.as_list()[-1]

    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

In [0]:
adam_optim = Adam(lr=0.01, beta_1=0.9, beta_2=0.999)


anchor_input = Input((224,224, 3), name='anchor_input')
positive_input = Input((224,224, 3), name='positive_input')
negative_input = Input((224,224, 3), name='negative_input')

Shared_DNN = create_base_network((224,224,3))

encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)

merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')
print(merged_vector)
model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.summary()
model.compile(loss=triplet_loss, optimizer=adam_optim)

def someMethodToLoadImages(files,batch_start,limit):
  arr=[]
  for i in range(batch_start,limit):
    img=cv2.imread(files[i],0)
    img=np.array(img)
    img=np.resize(img  ,(224,224,3))
    arr.append(img)
  return arr

def imageLoader(train_positive,train_negative, train_anchor, batch_size=64):
    
    L = len(train_positive)
    while True:
      batch_start = 0
      batch_end = batch_size
      X=[]
      print(batch_start)
      while batch_start < L:
        limit = min(batch_end, L)
        A = preprocess_input(np.array(someMethodToLoadImages(train_anchor,batch_start,limit)))
        P = preprocess_input(np.array(someMethodToLoadImages(train_positive,batch_start,limit)))
        N = preprocess_input(np.array(someMethodToLoadImages(train_negative,batch_start,limit)))
        
        X=[A,P,N]
        Y = np.empty((np.array(A).shape[0],3072))
        Y = np.array(Y)
        yield (X,Y) 
        batch_start += batch_size   
        batch_end += batch_size
              

Tensor("merged_layer_6/concat:0", shape=(?, 384), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_13 (Model)                (None, 128)          27625920    anchor_input[0][0]               
                                            

#CALLBACKS




In [0]:
def predict_prob(loaded_model, img1,img2):
  dummy=np.zeros((1,224,224,3))
  img1=preprocess_input(np.resize((np.array(cv2.imread(img1,0))),(1,224,224,3)))
  img2=preprocess_input(np.resize((np.array(cv2.imread(img2,0))),(1,224,224,3)))
  s=loaded_model.predict([img2, img1, dummy])

  s1=(s[0][128:256]).reshape(1,-1)

  s2=(s[0][0:128]).reshape(1,-1)
 
  similarity=metrics.pairwise.cosine_similarity(s1,s2)
  similarity=float(similarity)
  return similarity

In [0]:
alpha = 0.1
roc = []
class roc_test(callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
     if (epochs+1)%10 == 0:
       prob=[]
       true=[]
       for i in range(200):
         similarity=predict_prob(model, val_set[i][0],val_set[i][1])
         prob.append(similarity)
         true.append(val_set[i][2])
       roc.append(roc_auc_score(true, prob))
       l = len(roc)
       print("current roc score: ",roc[l-1])
       if(l>=2):
         if(roc[l-1]<roc[l-2]):
           lr = K.get_value(adam_optim.lr)
           K.set_value(adam_optim.lr, alpha * lr)
           print("learning rate:",lr,"new rate:",alpha*lr)
         elif(roc[l-1]>roc[l-2]): 
           print('roc improved .. saving...')
           filepath =  '/gdrive/My Drive/weights/' + str(epochs+1) 
           model_json = model.to_json()
           with open(filepath + '.json', 'w') as json_file:
            json_file.write(model_json)
           # serialize weights to HDF5
           model.save_weights(filepath + '.h5')
           print("Saved ", str(epochs+1)," epoch to disk")
          
          
roctest = roc_test()

In [0]:
batch_size=32
model.fit_generator(imageLoader(train_positive,train_negative, train_anchor,batch_size),steps_per_epoch=5, epochs=2000, verbose=1, callbacks = [roctest])

Epoch 1/20000

5/5 [==============================] - 5s 1s/step - loss: 0.4525
Epoch 2/2000
5/5 [==============================] - 5s 940ms/step - loss: 0.4450
Epoch 3/2000
5/5 [==============================] - 5s 951ms/step - loss: 0.4425
Epoch 4/2000
5/5 [==============================] - 5s 945ms/step - loss: 0.3975
Epoch 5/2000
5/5 [==============================] - 5s 953ms/step - loss: 0.4225
Epoch 6/2000
5/5 [==============================] - 5s 948ms/step - loss: 0.4150
Epoch 7/2000
5/5 [==============================] - 5s 948ms/step - loss: 0.4650
Epoch 8/2000
5/5 [==============================] - 5s 954ms/step - loss: 0.4200
Epoch 9/2000
5/5 [==============================] - 5s 954ms/step - loss: 0.4450
Epoch 10/2000
5/5 [==============================] - 5s 917ms/step - loss: 0.4225
current roc score:  0.4956931089743589
Epoch 11/2000
5/5 [==============================] - 5s 914ms/step - loss: 0.4225
Epoch 12/2000
5/5 [==============================] - 64s 13s/step - l

E0627 11:08:35.666271 139658024834944 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-102d514858f5>", line 2, in <module>
    model.fit_generator(imageLoader(train_positive,train_negative, train_anchor,batch_size),steps_per_epoch=5, epochs=2000, verbose=1, callbacks = [roctest])
  File "/usr/local/lib/python3.6/dist-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1418, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py", line 181, in fit_generator
    generator_output = next(output_generator)
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 685, in get
    inputs = self.queue.get(block=True).get()
  File "/usr/lib/python3.6/

KeyboardInterrupt: ignored

**Function to validate**

In [0]:
#@title validation function
def validate(model,val_set):
  c=0
  prob=[]
  true=[]
  for i in range(5):
    print(i, end=',')
    similarity=predict_prob(model, val_set[i][0],val_set[i][1])
    prob.append(similarity)
    true.append(val_set[i][2])
  roc = roc_auc_score(true, prob)
  print(roc)
  return roc
#validate(loaded_model, val_set)

In [0]:
# load json and create model
json_file = open('/gdrive/My Drive/weights/V2/12.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/gdrive/My Drive/weights/V2/12.h5")
print("Loaded model from disk")


In [0]:
loaded_model.summary()